In [ ]:
# ! pip install shapely networkx h3 geopandas folium duckdb numpy==1.26.4 scikit-learn==1.6.1 seaborn setuptools matplotlib rtree ipykernel pickle
! python -m pip install ipykernel -U --force-reinstall'

In [ ]:
! cd ../habit; pip install -e .

In [ ]:
!wget https://zenodo.org/records/17633792/files/final_syros.zip -O ../data/final_syros.zip
!unzip -o ../data/final_syros.zip -d ../data/
!head -n 5 ../data/final_syros/*.csv

In [2]:
import os
import pickle
from habit.imputer import HabitImputer

imputer=None
RESOLUTION=9
MAX_GAP=4


data_path = f"../data/final_syros/*.csv"
save_path = f'../models/habit_{RESOLUTION}_{MAX_GAP}.pickle';
# data_schema = { "id": "MMSI","lon": "LONGITUDE","lat": "LATITUDE","t": "TIMESTAMP","sog": "SOG","cog": "COG","trip":"TRIP" }
data_schema = { "id": "MMSI","lon": "LON","lat": "LAT","t": "TIMESTAMP","sog": "SOG","cog": "COG","trip":"MMSI" } # REPLACE MMSI WITH TRIP IF NEEDED


if not os.path.exists(save_path):    
    print("Creating new imputer")
    imputer = HabitImputer(resolution=RESOLUTION,schema=data_schema,max_gap=MAX_GAP)
    imputer.data_load(path=data_path)


    with open(save_path, 'wb') as handle:
        pickle.dump(imputer, handle, protocol=pickle.HIGHEST_PROTOCOL)
else :
    if os.path.getsize(save_path) > 0:    
        print("Loading existing imputer")
        with open(save_path, 'rb') as handle:
            imputer = pickle.load(handle)
    else:
        print("Corrupted pickle file")

Creating new imputer


In [4]:
import folium
import h3.api.numpy_int as h3

def add_h3_line_to_map(m, line, color, resolution):
    """
    Calculates H3 cells along a LineString and adds them + vertex markers to the map.
    
    Args:
        m (folium.Map): The map object to add features to.
        line (shapely.geometry.LineString): The line geometry.
        color (str): The color for the H3 hexagons.
        resolution (int): The H3 grid resolution (e.g., 9).
    """
    
    # 1. Extract coordinates from Shapely object
    coords = list(line.coords)
    hex_set = set()

    # 2. Iterate through segments to calculate H3 path
    for i in range(len(coords) - 1):
        p1 = coords[i]
        p2 = coords[i+1]
        
        # WKT is (Lon, Lat), H3 expects (Lat, Lon)
        start_hex = h3.latlng_to_cell(p1[1], p1[0], resolution)
        end_hex = h3.latlng_to_cell(p2[1], p2[0], resolution)
        
        try:
            # Interpolate hexes between points
            path = h3.grid_path_cells(start_hex, end_hex)
            hex_set.update(path)
        except Exception as e:
            # Fallback for errors (e.g., points too far apart)
            print(f"Segment {i} calculation error: {e}")
            hex_set.add(start_hex)
            hex_set.add(end_hex)

    # 3. Draw the Hexagons (using the 'color' argument)
    for h in hex_set:
        boundary = h3.cell_to_boundary(h) # Returns [(lat, lon), ...]
        folium.Polygon(
            locations=boundary,
            color=color,        # Use the function argument color
            weight=1,
            fill=True,
            fill_color=color,
            fill_opacity=0.2,
            tooltip=f"H3: {h}"
        ).add_to(m)

    # 4. Draw Markers at every vertex (using fixed Yellow as per snippet)
    # Note: We iterate through ALL coords to ensure the first point is also drawn
    for point in coords:
        lon, lat = point
        folium.CircleMarker(
            location=[lat, lon],
            radius=1,             # Radius 1 as requested
            color=color,       # Fixed contrast color for markers
            fill=True,
            fill_color=color,
            fill_opacity=1.0,
            popup=f"{lat:.5f}, {lon:.5f}"
        ).add_to(m)

In [5]:

# attr = (
#     '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> '
#     'contributors, &copy; <a href="https://cartodb.com/attributions">CartoDB</a>'
# )

gap1 = (24.95484012952308, 37.435773158346805,24.94894308913388,37.49993532157515)
gap2 = (24.960333761363813, 37.435773158346805, 24.978399055313172,37.42488753698335)
gap3 = (24.960333761363813, 37.435773158346805,25.153224624068056,37.528611280238685)




gaps = [gap1,gap2, gap3]

m = folium.Map(location=[37.435773158346805, 24.960333761363813],
            zoom_start=14,
            # atttr=attr,
            #tiles='CartoDB dark_matter'
            )

for (slon,slat,elon,elat) in gaps:
    print(f"Processing gap from ({slon}, {slat}) to ({elon}, {elat})")
  
    simplify_factor=0.002

    habit_line = imputer.fill_gap(slon,slat,elon,elat)
    habit_line_s = habit_line.simplify(simplify_factor, preserve_topology=False)
    habit_line_w = imputer.fill_gap_weighted(slon,slat,elon,elat)
    habit_line_w_s = habit_line_w.simplify(simplify_factor, preserve_topology=False)


    # 4. H3 Conversion Logic
    hex_set = set()



    folium.PolyLine( [tup[::-1] for tup in habit_line.coords], tooltip="habit",color="red").add_to(m)
    folium.PolyLine( [tup[::-1] for tup in habit_line_s.coords], tooltip="habit-s",color="red", dash_array='10').add_to(m)
    folium.PolyLine( [tup[::-1] for tup in habit_line_w.coords], tooltip="habit-w",color="yellow").add_to(m)
    folium.PolyLine( [tup[::-1] for tup in habit_line_w_s.coords], tooltip="habit-w-s",color="yellow", dash_array='10').add_to(m)


    add_h3_line_to_map(m, habit_line, "red", RESOLUTION)
    add_h3_line_to_map(m, habit_line_w, "yellow", RESOLUTION)

#
# m.save("h3_route_map.html")
m

Processing gap from (24.95484012952308, 37.435773158346805) to (24.94894308913388, 37.49993532157515)
Processing gap from (24.960333761363813, 37.435773158346805) to (24.978399055313172, 37.42488753698335)
Processing gap from (24.960333761363813, 37.435773158346805) to (25.153224624068056, 37.528611280238685)
